In [1]:
%load_ext autoreload

In [3]:
%autoreload

import h2o
import pandas as pd

from utilities import create_data as create
from utilities import create_delivery_file as delivery
from utilities import save_model as save
from h2o.automl import H2OAutoML

SyntaxError: invalid syntax (2820701172.py, line 6)

In [25]:
data = create.CreateTrainingData()
train = data.create_training_data(use_mean_values=True)
test = create.create_test_data(use_mean_values=True)
target = create.create_target_data()

In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /home/stinky/.local/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpa34kjdxg
  JVM stdout: /tmp/tmpa34kjdxg/h2o_stinky_started_from_python.out
  JVM stderr: /tmp/tmpa34kjdxg/h2o_stinky_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.4
H2O_cluster_version_age:,22 days
H2O_cluster_name:,H2O_from_python_stinky_po5nhg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.889 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [5]:
# Delete all rows with nans in pv_measurement column in train
# train = train[train['pv_measurement'].isna() == False]

X_train = h2o.H2OFrame(train)
y = 'pv_measurement'

X = X_train.columns
X.remove(y)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
autoML = H2OAutoML(max_models=30, seed=4, nfolds=5)
autoML.train(x=X, y=y, training_frame=X_train)

AutoML progress: |
13:39:17.187: XGBoost_1_AutoML_1_20231026_133916 [XGBoost def_2] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_1_AutoML_1_20231026_133916_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


████████████████████████
13:42:21.131: XGBoost_2_AutoML_1_20231026_133916 [XGBoost def_1] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_2_AutoML_1_20231026_133916_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


███████████████████████████
13:52:55.368: XGBoost_3_AutoML_1_20231026_133916 [XGBoost def_3] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_3_AutoML_1_20231026_133916_cv_1.  Details: ERRR on field: _response_column: Response contains missing va

key,value
Stacking strategy,cross_validation
Number of base models (used / total),5/30
# GBM base models (used / total),5/14
# DRF base models (used / total),0/2
# DeepLearning base models (used / total),0/13
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [7]:
best_model = autoML.leader
save.save_model_to_local_disk(best_model, 'h2o_mean')

In [8]:
# Remove features that are not in the training set
# test.fillna(0, inplace=True)
test_pd = test_mean[X]
test_h2o = h2o.H2OFrame(test_pd)

pred = autoML.predict(test_h2o)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [9]:
file = delivery.CreateDelivery(pred, 'mean_features_in_training_no_feature_engineering')
file.create_delivery_file(use_mean_values=True)